## Functies definiëren

In [1]:
# Importeren van benodigde libraries
import pandas as pd

# Functie om data in te laden
def load_data():
    # Lees de Excel bestanden in DataFrames
    df_omloopplanning = pd.read_excel('omloopplanning.xlsx', engine='openpyxl')
    df_dienstregeling = pd.read_excel('Connexxion data - 2024-2025.xlsx', engine='openpyxl')
    return df_omloopplanning, df_dienstregeling

def check_omloopplanning(omloop_df, dienst_df):
    # Voeg een nieuwe kolom toe om de correctheid te markeren
    omloop_df['correct'] = False

    # Filter alleen op dienstritten in omloopplanning
    dienst_ritten_omloop = omloop_df[omloop_df['activiteit'] == 'dienst rit']
    
    for idx, row in dienst_ritten_omloop.iterrows():
        # Filter voor de overeenkomstige rijen in de dienstregeling
        dienst_rows = dienst_df[
            (dienst_df['startlocatie'] == row['startlocatie']) &
            (dienst_df['eindlocatie'] == row['eindlocatie']) &
            (dienst_df['buslijn'] == row['buslijn'])
        ]
        
        for _, dienst_row in dienst_rows.iterrows():
            # Maak een vertrektijd datetime object
            vertrektijd = pd.to_datetime(f"{row['starttijd'].date()} {dienst_row['vertrektijd']}")
            
            # Controleer of de starttijd overeenkomt met de dienstregeling
            if vertrektijd == row['starttijd']:
                omloop_df.at[idx, 'correct'] = True
                break

    # Controleer of alle ritten in de dienstregeling aanwezig zijn in de omloopplanning
    dienst_df['found_in_omloop'] = False

    for idx, row in dienst_df.iterrows():
        omloop_rows = omloop_df[
            (omloop_df['startlocatie'] == row['startlocatie']) &
            (omloop_df['eindlocatie'] == row['eindlocatie']) &
            (omloop_df['buslijn'] == row['buslijn']) &
            (omloop_df['starttijd'].dt.time == pd.to_datetime(f"{row['vertrektijd']}").time()) &
            (omloop_df['activiteit'] == 'dienst rit')
        ]

        if not omloop_rows.empty:
            dienst_df.at[idx, 'found_in_omloop'] = True

    return omloop_df, dienst_df

## Controles uitvoeren

In [29]:
# Inladen van data
df_omloopplanning, df_dienstregeling = load_data()

# Converteer tijd kolommen naar datetime objecten voor makkelijke bewerking
df_omloopplanning['starttijd'] = pd.to_datetime(df_omloopplanning['starttijd datum'])
df_omloopplanning['eindtijd'] = pd.to_datetime(df_omloopplanning['eindtijd datum'])

# Voer de controle uit
df_omloopplanning, df_dienstregeling = check_omloopplanning(df_omloopplanning, df_dienstregeling)
# df_omloopplanning

### Deel 1: Resultaten van omloopplanning controleren

In [30]:
print('Resultaten van Omloopplanning Controle')
filtered_df = df_omloopplanning[df_omloopplanning['activiteit'] == 'dienst rit']
false_count = filtered_df['correct'].value_counts().get(False, 0)
print(f"Aantal onjuiste dienstritten in omloopplanning: {false_count}")
if false_count > 0:
    false_rows = filtered_df[filtered_df['correct'] == False]
    print("Onjuiste dienstritten in omloopplanning:")
    display(false_rows[['startlocatie', 'eindlocatie', 'starttijd', 'eindtijd', 'buslijn', 'correct']])

Resultaten van Omloopplanning Controle
Aantal onjuiste dienstritten in omloopplanning: 0


## Deel 2: Resultaten van dienstregeling controleren

In [4]:
print('\nResultaten van Dienstregeling Controle')
not_found_count = df_dienstregeling['found_in_omloop'].value_counts().get(False, 0)
print(f"Aantal dienstritten in dienstregeling die niet in omloopplanning zijn gevonden: {not_found_count}")
if not_found_count > 0:
    not_found_rows = df_dienstregeling[df_dienstregeling['found_in_omloop'] == False]
    print("\nDienstritten in dienstregeling die niet in de omloopplanning zijn gevonden:")
    display(not_found_rows[['startlocatie', 'eindlocatie', 'vertrektijd', 'buslijn']])


Resultaten van Dienstregeling Controle
Aantal dienstritten in dienstregeling die niet in omloopplanning zijn gevonden: 1

Dienstritten in dienstregeling die niet in de omloopplanning zijn gevonden:


,startlocatie,eindlocatie,vertrektijd,buslijn
327,ehvbst,foute test data,19:45,395


# Min/Max tijd checken

## Duur in min bepalen

In [6]:
import pandas as pd
# Eerst ervoor zorgen dat de kolommen 'starttijd' en 'eindtijd' in datetime geformatteerd zijn
df_omloopplanning['starttijd'] = pd.to_datetime(df_omloopplanning['starttijd'])
df_omloopplanning['eindtijd'] = pd.to_datetime(df_omloopplanning['eindtijd'])

# Nieuwe kolom 'duur' toevoegen die de verschil tussen eindtijd en starttijd aangeeft
df_omloopplanning['duur'] = df_omloopplanning['eindtijd'] - df_omloopplanning['starttijd']

# Als je de duur in minuten wilt omzetten
df_omloopplanning['duur_minuten'] = df_omloopplanning['duur'].dt.total_seconds() / 60

# Resultaat bekijken
display(df_omloopplanning[['starttijd', 'eindtijd', 'duur', 'duur_minuten']])

,starttijd,eindtijd,duur,duur_minuten
0,2024-08-29 05:03:00,2024-08-29 05:07:00,0 days 00:04:00,4.0
1,2024-08-29 05:07:00,2024-08-29 05:31:00,0 days 00:24:00,24.0
2,2024-08-29 05:31:00,2024-08-29 06:04:00,0 days 00:33:00,33.0
3,2024-08-29 06:04:00,2024-08-29 06:29:00,0 days 00:25:00,25.0
4,2024-08-29 06:29:00,2024-08-29 06:31:00,0 days 00:02:00,2.0
...,...,...,...,...
715,2024-08-29 17:06:00,2024-08-29 17:30:00,0 days 00:24:00,24.0
716,2024-08-29 17:30:00,2024-08-29 17:50:00,0 days 00:20:00,20.0
717,2024-08-29 14:02:00,2024-08-29 14:06:00,0 days 00:04:00,4.0
718,2024-08-29 14:06:00,2024-08-29 14:30:00,0 days 00:24:00,24.0


## Afstandsmatrix

In [27]:
df_afstandsmatrix = pd.read_excel('Connexxion data - 2024-2025.xlsx', engine='openpyxl', sheet_name='Afstandsmatrix')

# Voeg de kolommen samen en maak de nieuwe kolom 'code'
df_afstandsmatrix['code_afstand'] = df_afstandsmatrix['startlocatie'].astype(str) + '_' + \
                            df_afstandsmatrix['eindlocatie'].astype(str) + '_' + \
                            df_afstandsmatrix['buslijn'].astype(str)

# Controleer de eerste paar rijen van de DataFrame om zeker te zijn dat de nieuwe kolom is toegevoegd
display(df_afstandsmatrix)

,startlocatie,eindlocatie,min reistijd in min,max reistijd in min,afstand in meters,buslijn,code_afstand
0,ehvapt,ehvbst,21,23,10250,400.0,ehvapt_ehvbst_400.0
1,ehvbst,ehvapt,21,23,10708,400.0,ehvbst_ehvapt_400.0
2,ehvapt,ehvbst,22,25,9050,401.0,ehvapt_ehvbst_401.0
3,ehvbst,ehvapt,22,24,9003,401.0,ehvbst_ehvapt_401.0
4,ehvapt,ehvbst,20,20,8600,NaN,ehvapt_ehvbst_nan
5,ehvbst,ehvapt,20,20,8600,NaN,ehvbst_ehvapt_nan
6,ehvbst,ehvgar,4,4,1650,NaN,ehvbst_ehvgar_nan
7,ehvgar,ehvbst,4,4,1650,NaN,ehvgar_ehvbst_nan
8,ehvapt,ehvgar,20,20,9000,NaN,ehvapt_ehvgar_nan
9,ehvgar,ehvapt,20,20,9000,NaN,ehvgar_ehvapt_nan


## Daadwerkelijke check

In [28]:
import pandas as pd

# Samenvoegen van de DataFrames op basis van startlocatie, eindlocatie en buslijn
merged_df = df_omloopplanning.merge(df_afstandsmatrix, on=['startlocatie', 'eindlocatie', 'buslijn'], how='left')

# Voorwaarden controleren of de duur buiten de minimale en maximale reistijd valt
condition_out_of_bounds = (merged_df['duur_minuten'] < merged_df['min reistijd in min']) | (merged_df['duur_minuten'] > merged_df['max reistijd in min'])

# Nieuwe DataFrame met rijen waar de voorwaarde True is
df_out_of_bounds = merged_df[condition_out_of_bounds]

# Afdrukken of gebruiken van de resulterende DataFrame
print("Aantal regels buiten de minimale/maximale reistijd:", len(df_out_of_bounds))

# Unieke 'code_afstand' waarden in df_out_of_bounds
unieke_codes_domein = df_out_of_bounds['code_afstand'].unique()
print("Ritten buiten minimale/maximale reistijd:", unieke_codes_domein, "\n")

# Voorwaarden controleren of de duur onder de minimale reistijd ligt
condition_below_min = merged_df['duur_minuten'] < merged_df['min reistijd in min']

# Nieuwe DataFrame met rijen waar de voorwaarde True is
df_below_min = merged_df[condition_below_min]

# Afdrukken of gebruiken van de resulterende DataFrame
print("Aantal regels onder het minimale tijdsdomein:", len(df_below_min))

# Unieke 'code_afstand' waarden in df_below_min
unieke_codes_min = df_below_min['code_afstand'].unique()
print("Ritten onder de minimale reistijd:", unieke_codes_min, "\n")

if len(df_out_of_bounds) > 0:
    print("Details van rijen buiten de minimale/maximale reistijd:")
    display(df_out_of_bounds)

if len(df_below_min) > 0:
    print("Details van rijen onder de minimale reistijd:")
    display(df_below_min)

Aantal regels buiten de minimale/maximale reistijd: 151
Ritten buiten minimale/maximale reistijd: ['ehvbst_ehvapt_400.0' 'ehvapt_ehvbst_400.0'] 

Aantal regels onder het minimale tijdsdomein: 0
Ritten onder de minimale reistijd: [] 

Details van rijen buiten de minimale/maximale reistijd:


,Unnamed: 0,startlocatie,eindlocatie,starttijd,eindtijd,activiteit,buslijn,energieverbruik,starttijd datum,eindtijd datum,omloop nummer,correct,duur,duur_minuten,min reistijd in min,max reistijd in min,afstand in meters,code_afstand
9,9,ehvbst,ehvapt,2024-08-29 07:21:00,2024-08-29 07:45:00,dienst rit,400.0,12.8496,2024-08-29 07:21:00,2024-08-29 07:45:00,1,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
34,34,ehvbst,ehvapt,2024-08-29 16:51:00,2024-08-29 17:15:00,dienst rit,400.0,12.8496,2024-08-29 16:51:00,2024-08-29 17:15:00,1,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
72,72,ehvbst,ehvapt,2024-08-29 07:51:00,2024-08-29 08:15:00,dienst rit,400.0,12.8496,2024-08-29 07:51:00,2024-08-29 08:15:00,2,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
97,97,ehvbst,ehvapt,2024-08-29 17:21:00,2024-08-29 17:45:00,dienst rit,400.0,12.8496,2024-08-29 17:21:00,2024-08-29 17:45:00,2,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
133,133,ehvbst,ehvapt,2024-08-29 08:21:00,2024-08-29 08:45:00,dienst rit,400.0,12.8496,2024-08-29 08:21:00,2024-08-29 08:45:00,3,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,707,ehvbst,ehvapt,2024-08-29 12:54:00,2024-08-29 13:18:00,dienst rit,400.0,12.8496,2024-08-29 12:54:00,2024-08-29 13:18:00,19,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
709,709,ehvbst,ehvapt,2024-08-29 13:54:00,2024-08-29 14:18:00,dienst rit,400.0,12.8496,2024-08-29 13:54:00,2024-08-29 14:18:00,19,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
711,711,ehvbst,ehvapt,2024-08-29 15:06:00,2024-08-29 15:30:00,dienst rit,400.0,12.8496,2024-08-29 15:06:00,2024-08-29 15:30:00,19,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
715,715,ehvbst,ehvapt,2024-08-29 17:06:00,2024-08-29 17:30:00,dienst rit,400.0,12.8496,2024-08-29 17:06:00,2024-08-29 17:30:00,19,True,0 days 00:24:00,24.0,21.0,23.0,10708.0,ehvbst_ehvapt_400.0
